In [1]:
import gzip
import json
import pickle

import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/user_item_title_237742i.csv', index_col=0, parse_dates=['event_datetime'])
item_id_to_title = dict(zip(df['item_id'], df['title']))

with open('../arfifacts/user_oe.pkl', 'rb') as file:
    user_oe = pickle.load(file)
    
with open('../arfifacts/item_oe.pkl', 'rb') as file:
    item_oe = pickle.load(file)

# Week split

In [3]:
with gzip.open('../results/rutube_final_config_week_2024_06_22T11_32_42/predictions/original_sasrec.json.gz') as file:
    # File of json lines
    sasrec_predictions = [json.loads(line) for line in file]

with gzip.open('../results/rutube_final_config_week_2024_06_22T11_32_42/predictions/original_bert4rec.json.gz') as file:
    bert4rec_predictions = [json.loads(line) for line in file]

with gzip.open('../results/rutube_final_config_week_2024_06_22T11_32_42/train.json.gz') as file:
    train = [json.loads(line) for line in file]

# LOO split

In [15]:
with gzip.open('../results/rutube_final_config_loo_2024_06_22T13_24_47/predictions/original_bert4rec.json.gz') as file:
    sasrec_predictions = [json.loads(line) for line in file]

with gzip.open('../results/rutube_final_config_loo_2024_06_22T13_24_47/predictions/original_sasrec.json.gz') as file:
    bert4rec_predictions = [json.loads(line) for line in file]

with gzip.open('../results/rutube_final_config_loo_2024_06_22T13_24_47/train.json.gz') as file:
    train = [json.loads(line) for line in file]

# Evaluation

In [4]:
def get_top_n_recommendations(idx, data, train, n=10):
    record = data[idx]
    user_id = record['user_id']
    predictions = record['predictions'][:n]
    target = json.loads(record['test_actions'][0])['item_id']
    result = dict()
    result['target'] = item_oe.inverse_transform([[int(target)]])
    result['predictions'] = item_oe.inverse_transform(
        np.array(list(map(lambda x: x[0], predictions))).astype(int).reshape(-1, 1)
    )
    result['history'] =  item_oe.inverse_transform(
        np.array([record['item_id'] for record in train if record['user_id'] == user_id][-n:]).astype(int).reshape(-1, 1)
    )
    return result


def format_recommendations(recommendations):
    f = 'History:\n\t{}\n\nRecommendations:\n\t{}\n\nGround truth:\n\t{}'
    target = item_id_to_title[recommendations['target'].item()]
    predictions = list([item_id_to_title[item_id.item()] for item_id in recommendations['predictions']])
    history = list([item_id_to_title[item_id.item()] for item_id in recommendations['history']])
    return f.format(
        '\n\t'.join(history),
        '\n\t'.join(predictions),
        target
    )

In [6]:
for i in range(100):
    result = format_recommendations(get_top_n_recommendations(i, bert4rec_predictions, train))
    if 'матч' in result.lower():
        print(result)

History:
	Свадьба в фотостудии VS Классическая свадьба
	ЭКСТРЕМАЛЬНЫЕ ПРЯТКИ В ОГРОМНОМ ДОМЕ!
	Кухня | Сезон 2 | Серия 11

Recommendations:
	Украинский фронт - сражение в Авдеевке. Взяли террикон. 24 октября 2023
	Новые военные заводы России стали причиной истерики в НАТО
	Путин: глава Генштаба РФ рекомендовал пока оставить ВСУ в «Крынках»!
	Прямой эфир Матч ТВ
	Сводки с фронтов от Кота Костяна  08.11.23
	"Ели, пили, веселились... Или  сказ о том, как Путин ""устал""."
	Прямой эфир НТВ
	Шоу Воли, 33 выпуск
	"Отчего у либералов сморщенные лица, или ""Россия - первая в мире"""
	Воскресное утро с Котом Костяном

Ground truth:
	УДИВИТЕЛЬНЫЙ ЦИФРОВОЙ ЦИРК: ПИЛОТ ( English )
History:
	Время Ч: Илон Маск теряет спутники, Россия уничтожает Starlink
	Взять Киев или разгромить ВСУ? Полетаев о приоритетах России
	Почему на Украине зазвучали трезвые голоса и как Евросоюз морозит Зеленского – Лидов
	Куда ВСУ ударят в следующий раз? Кто под наибольшей угрозой и сколько ещё будет длиться СВО - Рожин
